In [1]:
#!/usr/bin/python
# -*- coding: latin-1 -*-

import inspect, os, sys

try :
    import pywikibot as pb
    from pywikibot.specialbots import UploadRobot

except :
    current_folder = os.path.realpath(os.path.abspath(os.path.split(inspect.getfile(inspect.currentframe()))[0]))
    folder_parts = current_folder.split(os.sep)
    pywikibot_folder = os.sep.join(folder_parts[:-1])

    if current_folder not in sys.path:
        sys.path.insert(0, current_folder)
    if pywikibot_folder not in sys.path:
        sys.path.insert(0, pywikibot_folder)

    import pywikibot as pb
    from pywikibot.specialbots import UploadRobot

from http_request_randomizer.requests.proxy.requestProxy import RequestProxy

from bs4 import BeautifulSoup
from mako.template import Template
import requests
from requests.compat import quote

from PIL import Image
from io import BytesIO
import re

import dateparser as dp
import datetime

commons_site = pb.Site("commons", "commons")

In [2]:
from modules.wmtools import is_commons_file, get_hash

In [16]:
cwd = os.getcwd()

images_directory = os.path.join(cwd, 'images')
if not os.path.exists(images_directory):
    os.makedirs(images_directory)

In [3]:
req_proxy = RequestProxy()

2018-01-19 23:54:22,057 root   DEBUG    === Initialized Proxy Parsers ===
2018-01-19 23:54:22,058 root   DEBUG    	 FreeProxy Parser of 'http://free-proxy-list.net' with required bandwidth: '150' KBs
2018-01-19 23:54:22,059 root   DEBUG    	 ProxyForEU Parser of 'http://proxyfor.eu/geo.php' with required bandwidth: '1.0' KBs
2018-01-19 23:54:22,060 root   DEBUG    	 RebroWeebly Parser of 'http://rebro.weebly.com' with required bandwidth: '150' KBs
2018-01-19 23:54:22,060 root   DEBUG    	 SemairProxy Parser of 'https://premproxy.com/list/' with required bandwidth: '150' KBs
2018-01-19 23:54:22,061 root   DEBUG    =================================
2018-01-19 23:54:22,544 http_request_randomizer.requests.parsers.FreeProxyParser DEBUG    Proxy Invalid: <zip object at 0x7f338ff3e408>
2018-01-19 23:54:22,960 http_request_randomizer.requests.parsers.ProxyForEuParser DEBUG    Proxy with low bandwidth: 0.178
2018-01-19 23:54:22,961 http_request_randomizer.requests.parsers.ProxyForEuParser DEBU

In [4]:
url = 'http://www.lamoncloa.gob.es/multimedia/galeriasfotograficas/presidente/Paginas/2015/280315galeriapompidou.aspx'
excluded = [3, 4]
categories = ['Alain Seban', 
              '2015 in Málaga', 
              'Mariano Rajoy in 2015', 
              'Centre Pompidou Málaga', 
              'Fleur Pellerin',
              'Francisco de la Torre']
cat_string = '\n'.join(['[[Category:'+i+']]' for i in categories])

In [5]:
while True:
    request = req_proxy.generate_proxied_request(url)
    if request is not None:
        #print("{0}".format(u''.join(request.text).encode('utf-8')))
        response = request.text
        break
    else :
        print ('Trying')

soup = BeautifulSoup(response, 'html.parser')

2018-01-19 23:54:36,172 root   DEBUG    Using headers: {'Connection': 'close', 'User-Agent': 'Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_6; es-es) AppleWebKit/533.20.25 (KHTML, like Gecko) Version/5.0.4 Safari/533.20.2'}
2018-01-19 23:54:36,174 root   DEBUG    Using proxy: 120.52.21.132:8082 | ProxyForEU
2018-01-19 23:55:06,210 root   DEBUG    Proxy unreachable - Removed Straggling proxy: 120.52.21.132:8082 | ProxyForEU PL Size = 747
2018-01-19 23:55:06,213 root   DEBUG    Using headers: {'Connection': 'close', 'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; zh-CN; rv:1.9.1.2) Gecko/20090729 Firefox/3.5.2 (.NET CLR 3.5.30729'}
2018-01-19 23:55:06,215 root   DEBUG    Using proxy: 177.234.7.66:65301 | FreeProxy


Trying


2018-01-19 23:55:36,251 root   DEBUG    Proxy unreachable - Removed Straggling proxy: 177.234.7.66:65301 | FreeProxy PL Size = 746
2018-01-19 23:55:36,253 root   DEBUG    Using headers: {'Connection': 'close', 'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.0; en-GB; rv:1.9.1.10) Gecko/20100504 Firefox/3.5.10 GTB7.0 ( .NET CLR 3.5.30729'}
2018-01-19 23:55:36,255 root   DEBUG    Using proxy: 77.95.3.43:8080 | Samair


Trying
RR Status 200


In [6]:
title = soup.find_all("h1", class_="contenidoTitulo")[0].get_text().strip().replace(':', ' - ').replace('  ', ' ')
#title = u'Segundo Gobierno de Rodríguez Zapatero (IX Legislatura) 2010'
title

'Mariano Rajoy inaugura el Centro Pompidou Málaga'

In [7]:
try:
    pub_date_sp = ' '.join(soup.find_all("div", class_="contenidoFecha")[0].get_text().strip().split(' ')[-4:])
    parsed_pub_date = dp.parse(pub_date_sp)
    pub_date = parsed_pub_date.strftime('%Y-%m-%d')
except :
    pub_date = None

pub_date

'2015-03-19'

In [8]:
p = re.compile("gFIO\[\d{1,2}\] = \\'(.*)\\';")
image_list = ['http://www.lamoncloa.gob.es' + url for url in re.findall(p, response)]
image_list

['http://www.lamoncloa.gob.es/multimedia/fotos/presidente/PublishingImages/2015/280315FotoFamiliaPompidou.JPG',
 'http://www.lamoncloa.gob.es/multimedia/fotos/presidente/PublishingImages/2015/280315FotoFamiliaPompidou2.JPG',
 'http://www.lamoncloa.gob.es/multimedia/fotos/presidente/PublishingImages/2015/280315Pompidou4.JPG',
 'http://www.lamoncloa.gob.es/multimedia/fotos/presidente/PublishingImages/2015/280315Pompidou3.JPG',
 'http://www.lamoncloa.gob.es/multimedia/fotos/presidente/PublishingImages/2015/280315Pompidou5.JPG']

In [9]:
p = re.compile("gFT\[\d{1,2}\] = \\'(.*)\\';")
description_list = [desc.strip().rstrip('.') for desc in re.findall(p, response)]
description_list

['Mariano Rajoy, en la inauguración del Centro Pompidou Málaga, con Francisco de la Torre, alcalde de Málaga; Alain Seban, presidente del Centre Pompidou; y Fleur Pellerin, ministra de Cultura y Comunicación de Francia',
 'Mariano Rajoy, en la inauguración del Centro Pompidou Málaga, con Francisco de la Torre, alcalde de Málaga; Alain Seban, presidente del Centre Pompidou; y Fleur Pellerin, ministra de Cultura y Comunicación de Francia',
 'Mariano Rajoy, en la inauguración del Centro Pompidou Málaga, con Francisco de la Torre, alcalde de Málaga; Alain Seban, presidente del Centre Pompidou; y Fleur Pellerin, ministra de Cultura y Comunicación de Francia',
 'Mariano Rajoy, en la inauguración del Centro Pompidou Málaga, con Francisco de la Torre, alcalde de Málaga; Alain Seban, presidente del Centre Pompidou; y Fleur Pellerin, ministra de Cultura y Comunicación de Francia',
 'Mariano Rajoy saluda al actor Antonio Banderas durante la inauguración del Centro Pompidou Málaga']

In [10]:
p = re.compile("gFA\[\d{1,2}\] = \\'(.*)\\';")
author_list = [author.strip() for author in re.findall(p, response)]
author_list

['Pool Moncloa',
 'Pool Moncloa',
 'Pool Moncloa',
 'Pool Moncloa',
 'Pool Moncloa']

In [11]:
truth_list = ['Pool' in author for author in author_list]
truth_list

[True, True, True, True, True]

In [13]:
template = u"""=={{int:filedesc}}==
{{Information
|description={{es|1=${description}}}
|date=${date}
|source=[${url} Galería fotográfica: ${title}]
|author=Ministerio de la Presidencia. Gobierno de España (${author})
|permission=
|other versions=
}}

=={{int:license-header}}==
{{LicenseReview}}
{{attribution-La Moncloa}}

[[Category:Files uploaded by User:Discasto]]
${cat_string}"""

vars = {
    "url": url,
    "description": None,
    "author": None,
    "date": pub_date,
    "title": title,
    "cat_string": cat_string
}

In [21]:
used_names = []

counter = 1
for i, image in enumerate(image_list):
    # If the image is excluded, skip
    if i in excluded:
        print ("Image excluded. Skipping")
        continue
        
    # First, the image is downloaded and stored
    image_url = quote(image.encode('utf-8'), ':/')
    image_name = '{} {:02d}.jpg'.format(title.replace(':', ' -').replace('  ', ' '), counter)
    counter += 1
    image_path = os.path.join(images_directory, image_name)
    while True:
        print ('Trying download')
        img_request = req_proxy.generate_proxied_request(image_url)
        if img_request is not None:
            image = Image.open(BytesIO(img_request.content))
            image.save(image_path)
            print ('Image accessed. Going to next image')
            break
        else :
            print ('Still trying download')

    # If the image is already in Commons, skip
    if is_commons_file(get_hash(image_path)) :
        print ("Image already in commons. Skipping")
        os.remove(image_path)
        continue

    # If the image name is already in commons, find a new name
    if pb.Page(commons_site, image_name, ns=6).exists():
        print ("Image name ({0}) already used in Commons".format(image_name))
        used_names.append(image_name)
        
    while True:
        if image_name in used_names :
            # Finding a new name
            image_subject = '.'.join(image_name.split('.')[:-1])
            image_extension = 'jpg'
            p = re.compile('(.*) ([0-9]{2}\.jpg)')
            m = p.match(image_name)
            if m is None:
                image_name = image_subject + ' 01.' + image_extension
            else :
                counter = int(m.group(2)[:2]) + 1
                image_name = '{} {:02d}.{}'.format(m.group(1), counter, image_extension)

            if pb.Page(commons_site, image_name, ns=6).exists():
                print ("Image name ({0}) already used in Commons. Finding a new name".format(image_name))
                used_names.append(image_name)
        else :
            print ("Preparing to upload image with name {0}".format(image_name))
            used_names.append(image_name)
            break

    # image upload
    vars["description"] = description_list[i]
    vars["author"] = author_list[i]
    t = Template(template)
    _text = t.render(**vars)

    bot = UploadRobot(image_path,
                      description = _text,
                      useFilename = image_name,
                      keepFilename = True,
                      verifyDescription = False,
                      ignoreWarning = True,
                      targetSite = commons_site)
    bot.run()

    os.remove(image_path)

2018-01-20 00:13:50,700 root   DEBUG    Using headers: {'Connection': 'close', 'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.1.6) Gecko/20091201 MRA 5.5 (build 02842) Firefox/3.5.6 (.NET CLR 3.5.30729'}
2018-01-20 00:13:50,701 root   DEBUG    Using proxy: 109.224.60.178:80 | Samair


Trying download
RR Status 200


2018-01-20 00:14:13,489 pywiki VERBOSE  Found 1 commons:commons processes running, including this one.
2018-01-20 00:14:13,493 requests_oauthlib.oauth1_auth DEBUG    Signing request <PreparedRequest [POST]> using client <Client client_key=0a73e346a40b07262b6e36bdba01cba4, client_secret=****, resource_owner_key=5ca4fafa5dd85bb0ecf516d96ad6bf07, resource_owner_secret=****, signature_method=HMAC-SHA1, signature_type=AUTH_HEADER, callback_uri=None, rsa_key=None, verifier=None, realm=None, encoding=utf-8, decoding=utf-8, nonce=None, timestamp=None>
2018-01-20 00:14:13,494 requests_oauthlib.oauth1_auth DEBUG    Including body in call to sign: True
2018-01-20 00:14:13,496 oauthlib.oauth1.rfc5849 DEBUG    Collected params: [('oauth_nonce', '35344651745624642931516407253'), ('oauth_timestamp', '1516407253'), ('oauth_version', '1.0'), ('oauth_signature_method', 'HMAC-SHA1'), ('oauth_consumer_key', '0a73e346a40b07262b6e36bdba01cba4'), ('oauth_token', '5ca4fafa5dd85bb0ecf516d96ad6bf07'), ('gaifrom

Image accessed. Going to next image
[[commons:File:Mariano Rajoy inaugura el Centro Pompidou Málaga 00.jpg]]
Image already in commons. Skipping
Trying download


2018-01-20 00:14:43,650 root   DEBUG    Proxy unreachable - Removed Straggling proxy: 37.29.82.115:65103 | FreeProxy PL Size = 734
2018-01-20 00:14:43,653 root   DEBUG    Using headers: {'Connection': 'close', 'User-Agent': 'Mozilla/5.0 (X11; U; Linux x86_64; en-US; rv:1.9.2.3) Gecko/20100524 Firefox/3.5.'}
2018-01-20 00:14:43,653 root   DEBUG    Using proxy: 110.37.216.6:8080 | FreeProxy


...

2018-01-20 00:15:59,136 oauthlib.oauth1.rfc5849 DEBUG    Collected params: [('oauth_nonce', '130894267453515606941516407359'), ('oauth_timestamp', '1516407359'), ('oauth_version', '1.0'), ('oauth_signature_method', 'HMAC-SHA1'), ('oauth_consumer_key', '0a73e346a40b07262b6e36bdba01cba4'), ('oauth_token', '5ca4fafa5dd85bb0ecf516d96ad6bf07'), ('iiprop', 'timestamp|user|comment|url|size|sha1|mime|metadata|archivename'), ('titles', 'File:Mariano Rajoy inaugura el Centro Pompidou Málaga 03.jpg'), ('prop', 'imageinfo'), ('action', 'query'), ('indexpageids', ''), ('continue', ''), ('iilimit', '500'), ('meta', 'userinfo'), ('uiprop', 'blockinfo|hasmsg'), ('maxlag', '5'), ('format', 'json')]
2018-01-20 00:15:59,137 oauthlib.oauth1.rfc5849 DEBUG    Normalized params: action=query&continue=&format=json&iilimit=500&iiprop=timestamp%7Cuser%7Ccomment%7Curl%7Csize%7Csha1%7Cmime%7Cmetadata%7Carchivename&indexpageids=&maxlag=5&meta=userinfo&oauth_consumer_key=0a73e346a40b07262b6e36bdba01cba4&oauth_nonce

Preparing to upload image with name Mariano Rajoy inaugura el Centro Pompidou Málaga 03.jpg


2018-01-20 00:15:59,236 requests_oauthlib.oauth1_auth DEBUG    Including body in call to sign: False
2018-01-20 00:15:59,237 oauthlib.oauth1.rfc5849 DEBUG    Collected params: [('oauth_nonce', '170170319095256804181516407359'), ('oauth_timestamp', '1516407359'), ('oauth_version', '1.0'), ('oauth_signature_method', 'HMAC-SHA1'), ('oauth_consumer_key', '0a73e346a40b07262b6e36bdba01cba4'), ('oauth_token', '5ca4fafa5dd85bb0ecf516d96ad6bf07')]
2018-01-20 00:15:59,238 oauthlib.oauth1.rfc5849 DEBUG    Normalized params: oauth_consumer_key=0a73e346a40b07262b6e36bdba01cba4&oauth_nonce=170170319095256804181516407359&oauth_signature_method=HMAC-SHA1&oauth_timestamp=1516407359&oauth_token=5ca4fafa5dd85bb0ecf516d96ad6bf07&oauth_version=1.0
2018-01-20 00:15:59,239 oauthlib.oauth1.rfc5849 DEBUG    Normalized URI: https://commons.wikimedia.org/w/api.php
2018-01-20 00:15:59,240 oauthlib.oauth1.rfc5849 DEBUG    Signing: signature base string: POST&https%3A%2F%2Fcommons.wikimedia.org%2Fw%2Fapi.php&oauth_

Upload successful.
2018-01-20 00:16:01,808 pywiki INFO     Upload successful.
Upload of Mariano Rajoy inaugura el Centro Pompidou Málaga 03.jpg successful.
2018-01-20 00:16:01,809 pywiki INFO     Upload of Mariano Rajoy inaugura el Centro Pompidou Málaga 03.jpg successful.

1 pages read
1 pages written
2018-01-20 00:16:01,812 pywiki INFO     
1 pages read
1 pages written
Script terminated successfully.
2018-01-20 00:16:01,812 pywiki INFO     Script terminated successfully.


Image excluded. Skipping
Image excluded. Skipping
